In [28]:
import os
import sys
import warnings

import pandas as pd
import numpy as np
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


warnings.filterwarnings('ignore')

In [29]:
df = pd.read_csv('datasets/amazon.csv')
df.head()

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [30]:
df.isnull().sum()

reviewText    0
Positive      0
dtype: int64

In [31]:
df.dtypes

reviewText    object
Positive       int64
dtype: object

In [32]:
df.shape

(20000, 2)

In [33]:
def lower_case(text):
    return text.lower()

In [34]:
df.head()

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [35]:
df.reviewText = df.reviewText.apply(lower_case)

In [36]:
df.head()

,reviewText,Positive
0,this is a one of the best apps acording to a b...,1
1,this is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"this is a silly game and can be frustrating, b...",1
4,this is a terrific game on any pad. hrs of fun...,1


In [37]:
def clean_unique(text):
    uniques = re.compile('[^\w\s]')
    return re.sub(uniques, ' ', text)

In [38]:
df = df.sort_values(by=['reviewText', 'Positive'], ascending=True)
df_sample_text = df['reviewText'][26]
df_sample_text_clean = clean_unique(df_sample_text)
print(df_sample_text)
print(df_sample_text_clean)


this is the game124 to get. those other bad comments r stupid. this app doesn't need wifi.  i like this app a lot its free and who doesn't love angry birds!!!
this is the game124 to get  those other bad comments r stupid  this app doesn t need wifi   i like this app a lot its free and who doesn t love angry birds   


In [39]:
df.reviewText = df.reviewText.apply(clean_unique)

In [40]:
df.reviewText[22]

'this is the best game they could think of i downloaded it and my fav  character is the original pig yay love it'

In [41]:
df_sample_text

"this is the game124 to get. those other bad comments r stupid. this app doesn't need wifi.  i like this app a lot its free and who doesn't love angry birds!!!"

In [42]:
def clean_stop_word(text):
    stop_word = stopwords.words('english')
    filter_without_stopwords = []

    for i in text.split():
        if i not in stop_word:
            filter_without_stopwords.append(i)

    return filter_without_stopwords

In [43]:
df.reviewText = df.reviewText.apply(clean_stop_word)

In [44]:
df.head()

,reviewText,Positive
6573,"[calculator, ultimate, outstanding, calculator...",1
17399,"[circlelauncher, clever, app, allows, one, ico...",1
6701,"[comiccat, first, last, comic, book, app, look...",1
14001,"[drawing, pad, exactly, drawing, app, kindle, ...",1
10262,"[farkle, fun, almost, addictive, dice, game, p...",1


In [45]:
df_test = df.copy()
df_test

,reviewText,Positive
6573,"[calculator, ultimate, outstanding, calculator...",1
17399,"[circlelauncher, clever, app, allows, one, ico...",1
6701,"[comiccat, first, last, comic, book, app, look...",1
14001,"[drawing, pad, exactly, drawing, app, kindle, ...",1
10262,"[farkle, fun, almost, addictive, dice, game, p...",1
...,...,...
9576,"[ziplist, handy, really, use, lot, phone, pull...",1
6175,"[zolmo, seems, give, tinkers, cu, supporting, ...",0
6624,"[zomg, love, game, sort, confuzling, parts, lo...",1
12180,"[app, stuff, new, amp, confusing, first, app, ...",1


In [46]:
df_smaple_test = df_test.reviewText[26]
df_smaple_test

['game124',
 'get',
 'bad',
 'comments',
 'r',
 'stupid',
 'app',
 'need',
 'wifi',
 'like',
 'app',
 'lot',
 'free',
 'love',
 'angry',
 'birds']

In [47]:
# p_stemmer = PorterStemmer()
# stemmed_word = []
# for i in df_smaple_test:
#     stemmed_word.append(p_stemmer.stem(i))

# stemmed_word

In [48]:
def stemming(text):
    p_stemmer = PorterStemmer()
    stemmed_word = []

    for i in text:
        stemmed_word.append(p_stemmer.stem(i))

    return stemmed_word

In [51]:
df.reviewText = df.reviewText.apply(stemming)

In [ ]:
df.head()